# Setting Up Work Environment

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
data_train = pd.read_csv("/kaggle/input/math482-2024-2025-1-hw-03/train.csv") #reading train data
data_test = pd.read_csv("/kaggle/input/math482-2024-2025-1-hw-03/test.csv") #reading test data

# Overview of the Data

**Training Data**

The training dataset contains 30,000 rows and 21 features.

**Features**:  Include abstract column names that suggest no information about the data. 8 of the 21 features have categorical values and the rest has numerical values.

**Target Variable**:  Numerical values.

**Missing Values**:  All features have missing values. Number of the missing rows is <3000 for each row. 

In [ ]:
data_train.info()

In [ ]:
# Check for missing values in the dataset
missing_values = data_train.isnull().sum()

# Display only columns with missing values
missing_columns = missing_values[missing_values > 0]

if missing_columns.empty:
    print("No missing values in the dataset.")
else:
    print("Missing values found:")
    print(missing_columns)

**Test Data**

The test dataset contains 10,000 rows and 21 features and there are missing values in each feature. It matches the structure of the training data.

In [ ]:
data_test.info()

In [ ]:
# Check for missing values in the dataset
missing_values = data_test.isnull().sum()

# Display only columns with missing values
missing_columns = missing_values[missing_values > 0]

if missing_columns.empty:
    print("No missing values in the dataset.")
else:
    print("Missing values found:")
    print(missing_columns)

**Distribution of the Features**

I will use histograms to see the distribution of the features 

In [ ]:
# Drop 'id' column as it is not a feature
#data_train = data_train.drop(columns=['id'])

# Separate numeric and categorical features
numeric_features = data_train.select_dtypes(include=['number']).columns.tolist()
categorical_features = data_train.select_dtypes(include=['object']).columns.tolist()
numeric_features.remove("target")


# Plot distributions for numeric features
# Drop 'id' column as it is not a feature
data_train = data_train.drop(columns=['id'])

# Separate numeric and categorical features
numeric_features = data_train.select_dtypes(include=['number']).columns.tolist()
categorical_features = data_train.select_dtypes(include=['object']).columns.tolist()
target = data_train["target"]
numeric_features.remove("target")

# Set the grid size (rows x cols)
n_cols = 2  # Number of columns in the grid
n_rows = -(-len(numeric_features) // n_cols)  # Calculate required rows (ceiling division)

# Create subplots for numeric features
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))  # Adjust figsize for better visibility

# Loop through each numeric feature to plot
for i, feature in enumerate(numeric_features):
    row, col = divmod(i, n_cols)  # Calculate row, col position in the grid
    ax = axes[row, col] if n_rows > 1 else axes[col]  # Handle 1-row case
    
    sns.histplot(data_train[feature], kde=True, bins=30, color='blue', edgecolor='black', alpha=0.7, ax=ax)
    ax.set_title(f'Distribution of {feature}')
    ax.set_xlabel(feature)
    ax.set_ylabel('Frequency')

# Remove any empty subplots (if features < total grid slots)
for i in range(len(numeric_features), n_rows * n_cols):
    fig.delaxes(axes.flatten()[i])

plt.tight_layout()
plt.show()

# Plot distributions for categorical features

# Set the grid size (rows x cols)
n_cols = 2  # Number of columns in the grid
n_rows = -(-len(categorical_features) // n_cols)  # Calculate required rows (ceiling division)

# Create subplots for categorical features
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))  # Adjust figsize for better visibility

# Loop through each categorical feature to plot
for i, feature in enumerate(categorical_features):
    row, col = divmod(i, n_cols)  # Calculate row, col position in the grid
    ax = axes[row, col] if n_rows > 1 else axes[col]  # Handle 1-row case
    
    sns.countplot(x=data_train[feature], palette='Set2', ax=ax)
    ax.set_title(f'Distribution of {feature}')
    ax.set_xlabel(feature)
    ax.set_ylabel('Count')
    ax.tick_params(axis='x', rotation=45)  # Rotate x-axis labels for readability

# Remove any empty subplots (if features < total grid slots)
for i in range(len(categorical_features), n_rows * n_cols):
    fig.delaxes(axes.flatten()[i])

plt.tight_layout()
plt.show()


We can see that numerical features are extremely skewed. These are not going to work well with the regression algorithms because regression algorithms assume the data is normally distributed. We could remove the outliers and consider the data where the majority of the values are gathered but doing so would remove most of the data because the rows where the outliers are may be different for each feature. We can also see that the feature_17 is categorical even though it has numerical values.

I will use the Yeo-Jhonson transformation method to reduce the skewness of the numerical features (except feature_17) and check their distributions again to see if it has worked. I use this method because it finds the best power transformation of the data and also handles negative and zero values that are present in the data.

In [ ]:
from sklearn.preprocessing import PowerTransformer


categorical_features.append("feature_17")
numeric_features.remove("feature_17")


# Initialize PowerTransformer for Yeo-Johnson
pt = PowerTransformer(method='yeo-johnson', standardize=False)  # standardize=False keeps the original scale

# Apply the transformation
data_train[numeric_features] = pt.fit_transform(data_train[numeric_features])

# Print the lambda values (one per feature)
print("Optimal lambda values for each feature:")
for feature, lambda_value in zip(numeric_features, pt.lambdas_):
    print(f"{feature}: {lambda_value}")

# Check skewness before and after transformation
from scipy.stats import skew
print("\nSkewness after Yeo-Johnson transformation:")
for feature in numeric_features:
    print(f"{feature}: {skew(data_train[feature].dropna())}")


Let's plot the numerical features again to see if the transformation helped.

In [ ]:
# Set the grid size (rows x cols)
n_cols = 2  # Number of columns in the grid
n_rows = -(-len(numeric_features) // n_cols)  # Calculate required rows (ceiling division)

# Create subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))  # Adjust figsize for better visibility

# Loop through each feature to plot
for i, feature in enumerate(numeric_features):
    row, col = divmod(i, n_cols)  # Calculate row, col position in the grid
    ax = axes[row, col] if n_rows > 1 else axes[col]  # Handle 1-row case
    
    sns.histplot(data_train[feature], kde=True, bins=30, color='blue', edgecolor='black', alpha=0.7, ax=ax)
    ax.set_title(f'Distribution of {feature}')
    ax.set_xlabel(feature)
    ax.set_ylabel('Frequency')

# Remove any empty subplots (if features < total grid slots)
for i in range(len(numeric_features), n_rows * n_cols):
    fig.delaxes(axes.flatten()[i])

plt.tight_layout()
plt.show()

**Plotting features**

I will plot all of the features in the training data with the target column to see the relation between them. I will use scatter plots with aggregated line plots for the numerical values and violin plots for the categorical values.

In [ ]:


# Set the grid size (rows x cols)
n_cols = 2  # Number of columns in the grid
n_rows = -(-len(numeric_features) // n_cols)  # Calculate required rows (ceiling division)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))  # Adjust figsize for better visibility

for i, feature in enumerate(numeric_features):
    row, col = divmod(i, n_cols)  # Calculate row, col position in the grid
    ax = axes[row, col] if n_rows > 1 else axes[col]  # Handle 1-row case
    
    # Sort by the feature
    sorted_data = data_train[[feature, "target"]].sort_values(by=feature)
    
    # Use a rolling window to compute the mean
    window_size = 100  # Number of points for rolling average
    sorted_data['rolling_mean'] = sorted_data["target"].rolling(window=window_size, min_periods=1).mean()
    
    ax.plot(sorted_data[feature], sorted_data['rolling_mean'], color='orange', label='Smoothed Target')
    ax.scatter(sorted_data[feature], sorted_data["target"], alpha=0.2, label='Raw Data', color='blue')
    ax.set_xlabel(feature)
    ax.set_ylabel("target")
    ax.set_title(f'Smoothed Line of {"target"} vs {feature} (Window={window_size})')
    ax.legend()

# Remove any empty subplots (if features < total grid slots)
for i in range(len(numeric_features), n_rows * n_cols):
    fig.delaxes(axes.flatten()[i])

plt.tight_layout()
plt.show()

# Plot violin plots for categorical features
# Set the grid size (rows x cols)
n_cols = 2  # Number of columns in the grid
n_rows = -(-len(categorical_features) // n_cols)  # Calculate required rows (ceiling division)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))  # Adjust figsize for better visibility

for i, feature in enumerate(categorical_features):
    row, col = divmod(i, n_cols)  # Calculate row, col position in the grid
    ax = axes[row, col] if n_rows > 1 else axes[col]  # Handle 1-row case
    
    sns.violinplot(x=data_train[feature], y=data_train["target"], data=data_train, ax=ax)
    ax.set_xlabel(feature)
    ax.set_ylabel("target")
    ax.set_title(f'Violin Plot of {"target"} by {feature}')
    ax.tick_params(axis='x', rotation=45)  # Rotate x-axis labels for better visibility

# Remove any empty subplots (if features < total grid slots)
for i in range(len(categorical_features), n_rows * n_cols):
    fig.delaxes(axes.flatten()[i])

plt.tight_layout()
plt.show()


# Data Preprocessing

**Handling missing values**

I will handle the missing values first because most of the preprocessing methods doesn't handle the missing values. I will use SimpleImputer for imputing both categorical and numeric data. But I will use "mean" as strategy for numeric data and "most_frequent" for categorical data.

In [ ]:
from sklearn.impute import SimpleImputer

# Separate the data into numeric and categorical parts
X_numeric = data_train[numeric_features]
X_categorical = data_train[categorical_features]

# **Impute numeric features** (mean or median)
num_imputer = SimpleImputer(strategy='mean')  # Change 'mean' to 'median' if needed
X_numeric_imputed = pd.DataFrame(num_imputer.fit_transform(X_numeric), columns=numeric_features)

# **Impute categorical features** (most frequent or constant)
cat_imputer = SimpleImputer(strategy='most_frequent')  # Use strategy='constant' for 'missing' or 'unknown'
X_categorical_imputed = pd.DataFrame(cat_imputer.fit_transform(X_categorical), columns=categorical_features)

# **Combine numeric and categorical parts**
data_train_imputed = pd.concat([X_numeric_imputed, X_categorical_imputed, data_train[['target']]], axis=1)


**Handling Non-Numeric Values**

We should turn every non-numeric data into numeric because most models and feature selection methods don't work with non-numeric data. Since we don't have any information about the features, I will use One-Hot-Encoding just be safe.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse= False )  # drop='first' to avoid multicollinearity
X_categorical_encoded = pd.DataFrame(encoder.fit_transform(data_train_imputed[categorical_features]))

**Feature Selection**

As we can see from the relation graphs above, some of the features are irrelevant to the target value. We should remove them to reduce the noise generated. I will use Value Inflation Variance(VIF) and correlation coefficient methods to find irrelevant numeric features and use mutual info regression method to find irrelevant categorical features.

In [ ]:
from sklearn.feature_selection import mutual_info_regression

# Calculate the mutual information score for each categorical feature
mi_scores = mutual_info_regression(X_categorical_encoded, data_train['target'])

# Print the scores for each feature
for i, feature in enumerate(categorical_features):
    print(f'Mutual Information Score for {feature}: {mi_scores[i]}')


We can conclude that all of the categorical data are irrelevant to the target. So I will exclude them.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


X_with_target = X_numeric_imputed.copy()
X_with_target['target'] = target  #

# 1. Calculate the correlation matrix (including the target)**
correlation_matrix = X_with_target.corr()

# 2. Plot a heatmap for the correlation matrix (including the target)**
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=False, cmap="coolwarm")
plt.title("Correlation Matrix (Including Target)")
plt.show()

# 3. Flatten the correlation matrix, keeping only the upper triangle (to avoid duplicates)**
correlation_pairs = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# 4. Unstack the upper triangle into a DataFrame**
correlated_features = correlation_pairs.unstack().dropna().abs().sort_values(ascending=False)

# 5. Filter pairs with high correlation (include target correlations)**
high_correlation_pairs = correlated_features[correlated_features > 0.1]

# 6. Display the most correlated feature pairs (include the target)**
print("\n--- Most Correlated Features ---\n")
print(high_correlation_pairs)

# 7. Show the correlation of each feature with the target**
target_correlations = correlation_matrix['target'].drop(labels=['target'], errors='ignore')  # Drop 'target' to avoid self-correlation
target_correlations = target_correlations.abs().sort_values(ascending=False)

print("\n--- Correlation of Features with Target ---\n")
print(target_correlations)


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor




# **2. Calculate VIF for each feature**
def calculate_vif(dataframe):
    """Calculate Variance Inflation Factor (VIF) for each feature in the DataFrame."""
    vif_data = pd.DataFrame()
    vif_data["Feature"] = dataframe.columns
    vif_data["VIF"] = [variance_inflation_factor(dataframe.values, i) for i in range(dataframe.shape[1])]
    return vif_data

# Get VIF for all features
vif_results = calculate_vif(X_numeric_imputed)

# **3. Filter features with VIF > 1 and display the VIF values**

print("\n--- Features with their VIF Values ---\n")
print(vif_results)


We can see that feature_08, feature_01, and feature_10 have very high vif values which indicates that they are the linear combination of multiple features. However, based on the correlation matrix above, we only have 3 relevant features in which feature_08 is the most important of them. So, I will not remove the feature_08 because I will only select three features (feature_08, feature_04, feature_15), and the other two features are not correlated to the feature_08. I know that because if they were, their vif value should have also been high.

In [ ]:
relevant_features = X_numeric_imputed[["feature_04","feature_08","feature_15"]].copy()


relevant_features.info()

I will preprocess the test data as well using the methods above

In [ ]:
test_data_numeric = data_test[numeric_features]

test_data_imputed = pd.DataFrame(num_imputer.fit_transform(test_data_numeric), columns=numeric_features)

test_data_final = test_data_imputed[["feature_04","feature_08","feature_15"]]

test_data_final.info()

**Not Handling of the Outliers**

I will not touch the outliers because I don't have any information about the data and they can be showing natural variance.

# Building the Model

**Linear Regression**

I already did the feature selection so I will not be optimizing parameters for linear regression. Because the parameters of linear regression are usually about optimizing the feature selection. However, I will be scaling the data since it is important for regression algorithms. Because the features with higher values would dominate the prediction of the linear regression.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(relevant_features)
X_test_scaled = scaler.transform(test_data_final)

# Initialize and train the Linear Regression model
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, target)

y_pred1 = linear_model.predict(X_test_scaled)

# Create the submission DataFrame
submission1 = pd.DataFrame({
    'id': data_test["id"],
    'target': y_pred1
})

# Save the submission file
submission1.to_csv('submission_linear.csv', index=False)

print("Submission file created: submission_linear.csv")

**Polynomial Regression**

Again, I will scale the data before linear regression so that large values don't dominate the prediction.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly_train = poly.fit_transform(relevant_features)
X_poly_test = poly.transform(test_data_final)


scaler_poly = StandardScaler()

X_train_scaled_poly = scaler_poly.fit_transform(X_poly_train)
X_test_scaled_poly = scaler_poly.transform(X_poly_test)

poly_model = LinearRegression()
poly_model.fit(X_train_scaled_poly, target)


# Make predictions
y_pred2 = poly_model.predict(X_test_scaled_poly)


# Create the submission DataFrame
submission2 = pd.DataFrame({
    'id': data_test["id"],
    'target': y_pred2
})

# Save the submission file
submission2.to_csv('submission_polynomial.csv', index=False)

print("Submission file created: submission_polynomial.csv")

**Support Vector Regression**

Support Vector Regression (SVR) relies on concepts like distances, margins, and kernels, which are directly affected by the scale of both the features and the target. So we have to scale both the features and target values unlike in linear regression. After making the prediction I will reverse the scaling for the predicted target so that the values are accurate. I will also use grid search method to find the best parameters for the SVR.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR


# Scale features (X) and target (y)
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# Scale the features (X)
X_train_scaled = scaler_X.fit_transform(relevant_features)
X_test_scaled = scaler_X.transform(test_data_final)

# Scale the target (y) (reshape needed for 1D arrays)
y_train_scaled = scaler_y.fit_transform(target.values.reshape(-1, 1)).ravel()  # y must be reshaped for scaling

# Define the parameter distribution
param_dist = {
    'C': [1, 10, 100],        # Regularization
    'epsilon': [0.1, 0.5, 1], # Margin of tolerance
    'kernel': ['rbf'],          # Kernel type
    'gamma': ['scale', 'auto'] # Kernel coefficient
}

# Initialize the SVR model
svr = SVR()

# Perform randomized search
random_search = RandomizedSearchCV(estimator=svr, param_distributions=param_dist, n_iter=5, scoring='neg_mean_squared_error', cv=3, verbose=2)
random_search.fit(X_train_scaled, y_train_scaled)

# Get the best parameters and best model
best_params = random_search.best_params_
best_model = random_search.best_estimator_

print("Best parameters found:", best_params)

# Make predictions using the best model
y_pred_scaled = best_model.predict(X_test_scaled)

# Reverse scaling for the target
y_pred3 = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()


# Create the submission file
submission3 = pd.DataFrame({
    'id': data_test["id"],
    'target': y_pred3
})
submission3.to_csv('submission_vector_optimized.csv', index=False)

print("Submission file created: submission_vector_optimized.csv")



**Neural Network**

I will scale the features and the target as similar to the above so that NN converges faster. I will also use the Keras tuner to automatically find the best parameters for the NN.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import keras_tuner as kt  # Import Keras Tuner


# Scale the features and target
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(relevant_features)
X_test_scaled = scaler_X.transform(test_data_final)

y_train_scaled = scaler_y.fit_transform(target.values.reshape(-1, 1)).ravel()

# Define the model-building function for Keras Tuner
def build_model(hp):
    model = Sequential()
    
    # First hidden layer: Tune the number of neurons
    model.add(Dense(
        units=hp.Int('units_layer_1', min_value=8, max_value=64, step=8),
        activation='relu',
        input_dim=3  # Input dimension (number of features)
    ))
    
    # Second hidden layer: Tune the number of neurons
    model.add(Dense(
        units=hp.Int('units_layer_2', min_value=8, max_value=64, step=8),
        activation='relu'
    ))
    
    # Output layer
    model.add(Dense(1))  # Single output for regression

    # Compile the model: Tune the learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        ),
        loss='mse',
        metrics=['mae']
    )
    return model

# Initialize the Keras Tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_loss',  # Optimize for validation loss
    max_epochs=50,  # Maximum number of epochs for training
    factor=3,  # Factor to reduce the number of configurations
    directory='my_tuner_dir',  # Directory to save tuning results
    project_name='NN_tuning'  # Name of the tuning project
)

# Perform hyperparameter search
tuner.search(X_train_scaled, y_train_scaled, epochs=50, validation_split=0.2, batch_size=32, verbose=1)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Best number of neurons in layer 1: {best_hps.get('units_layer_1')}")
print(f"Best number of neurons in layer 2: {best_hps.get('units_layer_2')}")
print(f"Best learning rate: {best_hps.get('learning_rate')}")

# Train the model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train_scaled, y_train_scaled, epochs=100, validation_split=0.2, batch_size=32, verbose=1)

# Evaluate the model on the test set
y_pred_scaled = best_model.predict(X_test_scaled)
y_pred4 = scaler_y.inverse_transform(y_pred_scaled).ravel()


# 8. Create the submission file
submission = pd.DataFrame({
    'id': data_test["id"],
    'target': y_pred4.ravel()
})
submission.to_csv('submission_NN_tuned.csv', index=False)

print("Submission file created: submission_NN_tuned.csv")

# Conclusion

We have a very high MSE score. That is probably because of the fact that I eliminated the majority of the data in the feature selection. But that was necessary because the eliminated data wasn't correlated with the target value at all. So we can say that the data provided in this task is not optimal for predicting the target